<a href="https://colab.research.google.com/github/Pramurta/Neural-Nets-Assignment-2/blob/main/Inception_v3_testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf

In [ ]:
import os

from tensorflow.keras import layers
from tensorflow.keras import Model

tf.keras.backend.clear_session()
  
from tensorflow.keras.applications.inception_v3 import InceptionV3

#Employing transfer learning by utilizing the pretrained weights of Inception V3 model up to layer "mixed9" trained on the imagenet dataset 
pre_trained_model_inception = InceptionV3(input_shape = (200, 200, 3), 
                                include_top = False, 
                                weights = "imagenet")

87924736/87910968 [==============================] - 1s 0us/step


In [ ]:
for layer in pre_trained_model_inception.layers:
    layer.trainable = False

last_layer = pre_trained_model_inception.get_layer('mixed9')
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output

last layer output shape:  (None, 4, 4, 2048)


In [ ]:
from tensorflow.keras.optimizers import Adam


# Flatten the output layer to 1 dimension
x = layers.Flatten()(last_output)
# Add a fully connected layer with 1024 hidden units and ReLU activation
x = layers.Dense(1024, activation='relu')(x)
# Add a dropout rate of 0.4
x = layers.Dropout(0.4)(x)
# Add a fully connected layer with 512 hidden units and ReLU activation   
x = layers.Dense(512, activation='relu')(x) 
# Add a dropout rate of 0.2
x = layers.Dropout(0.2)(x) 
# Add a fully connected layer with 256 hidden units and ReLU activation 
x = layers.Dense(256, activation='relu')(x)  
# Add a dropout rate of 0.1
x = layers.Dropout(0.1)(x)          
# Add a final softmax layer for classification
x = layers.Dense(10, activation='softmax')(x)           

model = Model(pre_trained_model_inception.input, x) 

lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-3,
    decay_steps=5,
    decay_rate=0.9)

opt = Adam(learning_rate=lr_schedule)

model.compile(optimizer = opt, 
              loss = 'categorical_crossentropy', 
              metrics = ['accuracy'])

checkpoint_filepath = '/content/checkpoint'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

early_stopping_callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=10)


In [ ]:
model.load_weights(checkpoint_filepath)

In [ ]:
!unzip 'test_images.zip'

In [ ]:
root_test_dir = '/content/content/test_images/'
classes = ['fabric', 'foliage', 'glass', 'leather', 'metal', 'paper', 'plastic', 'stone', 'water', 'wood']
correct_label = 0
total_count = 0
total_correct = 0
for cls in classes:
    cur_dir = root_test_dir+cls
    for filename in os.listdir(cur_dir):
        if not filename.endswith('.jpg'):
            continue
        img = image.load_img(cur_dir+'/'+filename, target_size=(200, 200))
        x=image.img_to_array(img)
        x=x/255
        x=np.expand_dims(x, axis=0)
        images = np.vstack([x])
        classes = model.predict(images, batch_size=10)
        pred = np.argmax(classes[0])
        if pred==correct_label:
            total_correct+=1
        total_count+=1
    correct_label+=1

print('Test Accuracy: '+str(total_correct/total_count))


Test Accuracy: 0.796875
